In [4]:
# Import relevant libraries to make HTTP requests and parse JSON response
import requests
import json
import csv
import io
import chembl_webresource_client as cwc
import pandas as pd

print("done")

done


In [18]:
# Set disease_id variable for desired disease
disease_id = "EFO_0005537"

# Build query string to get target information as well as count
query_string = """
query AssociatedTargets {
  disease(efoId: "disease_id") {
    id
    name
    associatedTargets(page: { size: 1000, index: 1 }) {
      rows {
        target {
          id
          approvedName
          dbXrefs{
              id
            }
        }
        score
      }
    }
  }
}
""".replace("disease_id", disease_id)

# Set variables object of arguments to be passed to endpoint
variables = {"efoId": disease_id}

# Set base URL of GraphQL API endpoint
base_url = "https://api.platform.opentargets.org/api/v4/graphql"

# Perform POST request and check status code of response
r = requests.post(base_url, json={"query": query_string, "variables": variables})
print(r.status_code)

#Transform API response from JSON into Python dictionary and print in console
api_response = json.loads(r.text)
print('done')
print(api_response)

200
done
{'data': {'disease': {'id': 'EFO_0005537', 'name': 'triple-negative breast cancer', 'associatedTargets': {'rows': [{'target': {'id': 'ENSG00000096717', 'approvedName': 'sirtuin 1', 'dbXrefs': [{'id': '14929'}, {'id': '4I5I'}, {'id': '4IF6'}, {'id': '4IG9'}, {'id': '4KXQ'}, {'id': '4ZZH'}, {'id': '4ZZI'}, {'id': '4ZZJ'}, {'id': '5BTR'}, {'id': 'CHEMBL4506'}, {'id': 'DB15493'}, {'id': 'DB02709'}, {'id': 'DB13978'}, {'id': 'R-HSA-3371453'}, {'id': 'R-HSA-400253'}, {'id': 'R-HSA-427359'}, {'id': 'R-HSA-9617629'}, {'id': 'R-HSA-9707616'}, {'id': 'IPR029035'}, {'id': 'IPR003000'}, {'id': 'IPR026591'}, {'id': 'IPR026590'}]}, 'score': 0.049293886578312164}, {'target': {'id': 'ENSG00000170145', 'approvedName': 'salt inducible kinase 2', 'dbXrefs': [{'id': '21680'}, {'id': 'CHEMBL5699'}, {'id': 'DB12010'}, {'id': 'IPR011009'}, {'id': 'IPR000719'}, {'id': 'IPR017441'}, {'id': 'IPR008271'}, {'id': 'IPR017090'}, {'id': 'IPR034672'}, {'id': 'IPR015940'}, {'id': 'SIDG34457'}]}, 'score': 0.04

In [19]:
a=0
#create list of target IDs with associated evidence scores by calling them from dictionary
target_info = api_response['data']['disease']['associatedTargets']['rows']
#print(target_info)


target_id_list = []
#pull IDs and scores from dictionary and add to new list
while a < len(target_info):
    print(a)

    for dict in target_info[a]['target']['dbXrefs']:
        #print(dict)
        # Get rid of all IDs that aren't CHEMBL IDs, then put CHEMBL ID in list to add to tuple
        newDict = {key:val for key, val in (dict).items() if "CHEMBL" in val}
        if len(newDict) == 0:
            continue
        chembl_id = newDict['id']
    target_id_list.append((target_info[a]['target'].get('id'), chembl_id, target_info[a]['score']))
    a+=1
print(target_id_list)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [21]:
from chembl_webresource_client.new_client import new_client

target = new_client.target
activity = new_client.activity

# Create targets_dict, which will be a dictionary containing target names, with compounds and compound data associated with that target.
targets_dict = {}

# Create darget_data, which will be a dictionary containing target names, with dataframes with compound data associated with that target
target_data_list = []

#For each target pulled from OpenTargets:
for target in target_id_list:
    target_data = {}
    print(target[0])
    # Get compound activitiy data from Chembl for each target, including only exact IC50 values. Sort by IC50 value.
    activities = activity.filter(target_chembl_id=target[1], standard_type="IC50", relation = '=').order_by('standard_value')
    #print(activities)
    
    compound_dictionary = {}
    
    #create seperate list for each type of data
    smiles_list = []
    standard_value_list = []
    compound_list = []
    type_list = []
    assays_list = []

    print(len(activities))
    
    for compound in activities:
        #print(compound)
        slimDict = {key:val for key, val in compound.items() if key == 'type' or key == 'canonical_smiles' or key == 'standard_value'}
        

        #slimDict = {key:val for key, val in compound.items() if key == 'value'}

        
        compound_id = compound['molecule_chembl_id']
        compound_list.append(compound_id)
        type_list.append(compound['type'])
        standard_value_list.append(compound['standard_value'])
        smiles_list.append(compound['canonical_smiles'])
        assays_list.append(compound['assay_chembl_id'])        
        #print(compound_id)
    
        compound_dictionary[compound['molecule_chembl_id']] = slimDict
        df = pd.DataFrame.from_dict(compound_dictionary)
    
    
    targets_dict[target[0]] = compound_dictionary

    
    to_pandas_dict = {"compound id": compound_list, "smiles": smiles_list, "type" : type_list, "standard value": standard_value_list, 'assay_id': assays_list}
    pandas_df = pd.DataFrame.from_dict(to_pandas_dict)
    f = open(target[0]+"dataframe.pickle", "wb")
    pickle.dump(pandas_df, f)
    f.close()
    
    #target_data['target_id'] = target[0]
    #target_data['data'] = pandas_df
    #target_data_list.append(target_data)

    


"""
import pickle

for target in target_data_list:
    print(target['target_id'])

    display(target['data'])
    f = open(target['target_id']+"dataframe.pickle", "wb")
    pickle.dump(target['data'], f)
    f.close()
    #display(target['data'])
"""




ENSG00000096717
896
ENSG00000170145
55
ENSG00000144810
55
ENSG00000112062
5378
ENSG00000105856
5378
ENSG00000071242
298
ENSG00000048740
298
ENSG00000161011
0
ENSG00000130208
0
ENSG00000162337
0
ENSG00000075218
0
ENSG00000187323
0
ENSG00000074410
83
ENSG00000185345
83
ENSG00000177556
83
ENSG00000187244
83
ENSG00000012061
82
ENSG00000233725
82
ENSG00000137033
0
ENSG00000261040
0
ENSG00000162733
273
ENSG00000147655
273
ENSG00000107562
0
ENSG00000070831
12
ENSG00000084207
144
ENSG00000170779
144
ENSG00000154277
20
ENSG00000146674
0
ENSG00000110713
0
ENSG00000151692
0
ENSG00000093072
0
ENSG00000133794
0
ENSG00000134013
417
ENSG00000113739
417
ENSG00000168961
10
ENSG00000148346
10
ENSG00000183671
0
ENSG00000091317
0
ENSG00000108244
0
ENSG00000091436
64
ENSG00000163848
64
ENSG00000134294
64
ENSG00000158769
64
ENSG00000001497
64
ENSG00000149269
281
ENSG00000154342
51
ENSG00000101447
51
ENSG00000197905
75
ENSG00000138185
7
ENSG00000198910
7
ENSG00000173110
0
ENSG00000257923
0
ENSG00000142871
0


'\nimport pickle\n\nfor target in target_data_list:\n    print(target[\'target_id\'])\n\n    display(target[\'data\'])\n    f = open(target[\'target_id\']+"dataframe.pickle", "wb")\n    pickle.dump(target[\'data\'], f)\n    f.close()\n    #display(target[\'data\'])\n'

In [25]:
big_target_data_list = [target for target in target_data_list if len(target['data']) >3000]
print(type(target_data_list[0]['data']))
print(big_target_data_list)

IndexError: list index out of range

ENSG00000198900


,compound id,smiles,type,standard value,assay_id
0,CHEMBL3968023,C[n+]1ccc2c3c(nc4cc5c(cc42)OCO5)-c2ccccc2-c31....,IC50,0.45,CHEMBL3867357
1,CHEMBL3914791,C[n+]1c2c3c(nc(-c4ccccc4)cc3c3ccccc31)-c1ccccc...,IC50,0.475,CHEMBL3867357
2,CHEMBL3896867,[Br-].c1ccc(C[n+]2ccc3c4c(nc5cc6c(cc53)OCO6)-c...,IC50,0.52,CHEMBL3867357
3,CHEMBL3889816,[Br-].c1ccc(C[n+]2ccc3c4c(nc5ccccc53)-c3ccccc3...,IC50,0.57,CHEMBL3867357
4,CHEMBL3984603,Cc1cc2c3c(nc4ccccc42)-c2ccccc2-c3[n+]1C.[I-],IC50,0.7,CHEMBL3867357
...,...,...,...,...,...
451,CHEMBL426092,O=C1Oc2ccc(O)cc2/C1=C\c1ccc(O)cc1,IC50,555000.0,CHEMBL827155
452,CHEMBL9164,CCCCCCC(=O)O[C@H](CC=C(C)C)C1=CC(=O)c2c(O)ccc(...,IC50,625000.0,CHEMBL663903
453,CHEMBL372372,COc1cc(/C=C2/C(=O)Oc3ccccc32)ccc1O,IC50,630000.0,CHEMBL827155
454,CHEMBL381195,COc1ccc(/C=C2/C(=O)Oc3ccccc32)cc1O,IC50,800000.0,CHEMBL827155


In [194]:
# Get TXT file of list of SMILES associated with target
for target in target_data_list:
    print(target['target_id'])
    smiles = target['data']['smiles']
    print(smiles)
    smiles_list = []
    for row in smiles:
        smiles_list.append(row)
    #print(smiles_list)

    smiles_string = str(smiles_list)
    smiles_string = smiles_string[1:-1]
    smiles_string = smiles_string.replace(",", "\n").replace("'", "").replace(" ", "")
    print(smiles_string)

    with open(target['target_id']+'_Smiles.txt','w') as data: 
        data.write(str(smiles_string))
        # Get TXT file of list of SMILES associated with target


ENSG00000184292
Series([], Name: smiles, dtype: float64)

ENSG00000198900
0      C[n+]1ccc2c3c(nc4cc5c(cc42)OCO5)-c2ccccc2-c31....
1      C[n+]1c2c3c(nc(-c4ccccc4)cc3c3ccccc31)-c1ccccc...
2      [Br-].c1ccc(C[n+]2ccc3c4c(nc5cc6c(cc53)OCO6)-c...
3      [Br-].c1ccc(C[n+]2ccc3c4c(nc5ccccc53)-c3ccccc3...
4           Cc1cc2c3c(nc4ccccc42)-c2ccccc2-c3[n+]1C.[I-]
                             ...                        
451                    O=C1Oc2ccc(O)cc2/C1=C\c1ccc(O)cc1
452    CCCCCCC(=O)O[C@H](CC=C(C)C)C1=CC(=O)c2c(O)ccc(...
453                   COc1cc(/C=C2/C(=O)Oc3ccccc32)ccc1O
454                   COc1ccc(/C=C2/C(=O)Oc3ccccc32)cc1O
455                       O=C1Oc2ccc(O)cc2/C1=C\c1ccccc1
Name: smiles, Length: 456, dtype: object
C[n+]1ccc2c3c(nc4cc5c(cc42)OCO5)-c2ccccc2-c31.[I-]
C[n+]1c2c3c(nc(-c4ccccc4)cc3c3ccccc31)-c1ccccc1-2.[I-]
[Br-].c1ccc(C[n+]2ccc3c4c(nc5cc6c(cc53)OCO6)-c3ccccc3-c42)cc1
[Br-].c1ccc(C[n+]2ccc3c4c(nc5ccccc53)-c3ccccc3-c42)cc1
Cc1cc2c3c(nc4ccccc42)-c2ccccc2-c3[n

In [217]:

smiles = "CC(O)C"
headers = 	{
	"API Key":"0794a3fb-40ff-4465-9b26-40bfad7ce84e",
	"Structure":smiles,
	"Search Type":'3',
	"Maximum Search Time":'60000',
	"Maximum Result Count":'10000',
	"Chemical Similarity Index":'0.9'
	}
url = "https://api.molport.com/api/chemical-search/search"

#r = requests.get("https://api.molport.com/api/molecule/load?molecule=2325030&apikey=880d8343-8ui2-418c-9g7a-68b4e2e78c8b")
r = requests.get(url = url, params = headers)
r.json()

{'Result': {'Status': 2, 'Message': 'Username or password is incorrect!'},
 'Data': {'Molecules': [], 'Version': 'v.1.0'}}

In [ ]:
r = requests.get("https://zinc20.docking.org/for_sale?CC(C)O")
print(r.status)

NameError: name 'compund_dictionary' is not defined